In [20]:
!cd drive/MyDrive/Network/
!ls

drive  sample_data


In [21]:
import networkx as nx
import random
import csv
from csv import writer
from csv import reader
from geopy.distance import great_circle
from numpy.lib.utils import source
import requests
import json
import numpy as np, numpy.random
from matplotlib import pylab
import matplotlib.pyplot as plt
import pandas as pd
import time 
import csv



In [22]:
def getNodes(number):
  arr=[]
  for i in range(number):
    arr+=[[i]]
  return arr
def randomList(size, sumt):
    arr = [1] * size;
    for i in range(sumt-size) :
        arr[random.randint(0, sumt) % size] += 1;
    return arr
def GetSources(sourceNodes,TotalCapacity):
    sources=[]
    sourcesCapacity = randomList(len(sourceNodes),TotalCapacity)
    for i in range(len(sourceNodes)):
        sources+=[sourceNodes[i]+[sourcesCapacity[i]]]
    print("Source Demand is: "+str(sum(sourcesCapacity)))
    return sources
def GetSinks(sinkDistr,TotalRequirement):
    sinkRequirement = randomList(len(sinkDistr),TotalRequirement)
    for i in range(len(sinkDistr)):
        sinkDistr[i]+=[sinkRequirement[i]]
    print("Sink Demand is: "+str(sum(sinkRequirement)))
    return sinkDistr
def distributeSourceSink(districts,NumberOfSources):
    sourceDistr=[]
    sinkDistr=[]
    for i in range(NumberOfSources):
        sourceDistr+=[districts[i]]
    for i in range(NumberOfSources,len(districts)):
        sinkDistr+=[districts[i]]
    return sourceDistr, sinkDistr
def getEdgesDummy(sources,sinks):
    edges=[]
    for source in sources:
        for sink in sinks:
            # if sink!=source:
            edges.append(source+sink+[random.randint(100,10000)])
    return edges

In [23]:
def modelSimplex(totalSources,totalSinks,edges):
  G = nx.DiGraph()
  demand=0
  for source in totalSources:
    G.add_node(source[0],type='source',demand=-1*int(source[1]))
    demand+=-1*int(source[1])
  for sink in totalSinks:
    G.add_node(sink[0],type='sink',demand=int(sink[1]))
    demand+=int(sink[1])
  for edge in edges:
    G.add_edge(edge[0], edge[2], weight=int(edge[4]) )
  print("Total Demand is: ",demand)
  return G


def modelMaxFlow(totalSources,totalSinks,edges):
  G = nx.DiGraph()
  G.add_node("Super-source",type='source')
  G.add_node("Super-sink",type='sink')
  for source in totalSources:
    G.add_node(source[0],type='source')
    G.add_edge("Super-source",source[0],capacity=source[1])
  for sink in totalSinks:
    G.add_node(sink[0],type='sink')
    G.add_edge(sink[0],"Super-sink",capacity=sink[1])
  for edge in edges:
    G.add_edge(edge[0], edge[2], weight=int(edge[4]) )
  return G


In [24]:
def printGraphStats(G1,withFig):
  if withFig==True:
    nx.draw_networkx(G1, with_labels = True)
    plt.show()
  print(G1.nodes())
  print(G1.edges())

In [ ]:
tempList=[100,200,300,400,500,600,700,800,900,1000,1200,1400,1600,1800,2000]
tempList2=[2000,2500,3000,3500,4000]

experiment=[]
for i in tempList2:
  # experiment+=[[50,i]]
  # experiment+=[[100,i]]
  experiment+=[[50,i]]
for i in tempList:
  experiment+=[[500,i]]
  # experiment+=[[400,i]]
experiment.sort()
print(experiment)
header = ['sources', 'sinks', 'Time-create-MCMF', 'Time-create-simplex', 'Time-run-MCMF', 'Time-run-simplex']
data=[]
# experiment=[[50,100],[100,300],[100,1200],[50,600],[200,600]]
with open('countries.csv', 'w', encoding='UTF8', newline='') as f:
  writer = csv.writer(f)
  writer.writerow(header)
for event in experiment:
  row=[]
  data=[]
  print("Event with Sources:",event[0],"Sinks:",event[1])
  row+=[event[0],event[1]]
  numberOfNodes=event[0]+event[1]
  TotalCapacity=10000
  TotalRequirement=10000
  nodes=getNodes(numberOfNodes)
  sources,sinks=distributeSourceSink(nodes,event[1])
  sources=GetSources(sources,TotalCapacity)
  sinks=GetSinks(sinks,TotalRequirement)
  edges=getEdgesDummy(sources,sinks)
  start = time.time()
  G1=modelMaxFlow(sources,sinks,edges)
  end = time.time()
  row+=[end-start]
  print("Time to create Max Flow Model:",end - start)
  start = time.time()
  G2=modelSimplex(sources,sinks,edges)
  end = time.time()
  row+=[end-start]
  print("Time to create Simplex Model:",end - start)
  start = time.time()
  mincostFlow = nx.max_flow_min_cost(G1, "Super-source", "Super-sink")
  # print("Cost is:", nx.cost_of_flow(G1, mincostFlow))
  end = time.time()
  row+=[end-start]
  print("Time to run Max Flow Model:",end - start)
  start = time.time()
  flowCost, flowDict = nx.network_simplex(G2)
  # print("Cost is:",flowCost)
  end = time.time()
  row+=[end-start]
  print("Time to run Simplex Model:",end - start)
  print("--------------------------------")
  data+=[row]
  print(row)
  with open('countries.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(data)


In [25]:
# numberOfNodes=1000
# TotalCapacity=10000
# TotalRequirement=10000
# nodes=getNodes(numberOfNodes)
# sources,sinks=distributeSourceSink(nodes,numberOfNodes//20)
# sources=GetSources(sources,TotalCapacity)
# sinks=GetSinks(sinks,TotalRequirement)
# edges=getEdgesDummy(sources,sinks)


Source Demand is: 10000
Sink Demand is: 10000


In [26]:
# print(nodes)
# print(sources)
# print(sinks)
# print(edges)

[[0], [1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11], [12], [13], [14], [15], [16], [17], [18], [19], [20], [21], [22], [23], [24], [25], [26], [27], [28], [29], [30], [31], [32], [33], [34], [35], [36], [37], [38], [39], [40], [41], [42], [43], [44], [45], [46], [47], [48], [49], [50, 14], [51, 8], [52, 9], [53, 10], [54, 10], [55, 16], [56, 8], [57, 14], [58, 10], [59, 13], [60, 6], [61, 9], [62, 9], [63, 10], [64, 6], [65, 11], [66, 8], [67, 14], [68, 4], [69, 15], [70, 9], [71, 10], [72, 13], [73, 7], [74, 14], [75, 11], [76, 10], [77, 11], [78, 13], [79, 9], [80, 12], [81, 12], [82, 14], [83, 9], [84, 15], [85, 8], [86, 5], [87, 10], [88, 12], [89, 9], [90, 13], [91, 11], [92, 12], [93, 8], [94, 15], [95, 17], [96, 12], [97, 11], [98, 14], [99, 14], [100, 7], [101, 5], [102, 9], [103, 12], [104, 7], [105, 9], [106, 14], [107, 3], [108, 13], [109, 11], [110, 14], [111, 15], [112, 14], [113, 10], [114, 15], [115, 10], [116, 12], [117, 7], [118, 11], [119, 13], [120, 13], [1

In [27]:
# G1=modelMaxFlow(sources,sinks,edges)
# printGraphStats(G1,False)

['Super-source', 'Super-sink', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215,

In [28]:
# start = time.time()
# mincostFlow = nx.max_flow_min_cost(G1, "Super-source", "Super-sink")
# print("Cost is:", nx.cost_of_flow(G1, mincostFlow))
# end = time.time()
# print(end - start)
# print(mincostFlow)

Cost is: 3034933
7.657419919967651
{'Super-source': {0: 199, 1: 184, 2: 206, 3: 219, 4: 199, 5: 198, 6: 201, 7: 190, 8: 174, 9: 209, 10: 222, 11: 193, 12: 160, 13: 203, 14: 204, 15: 225, 16: 219, 17: 193, 18: 190, 19: 197, 20: 209, 21: 203, 22: 203, 23: 197, 24: 216, 25: 220, 26: 198, 27: 207, 28: 191, 29: 219, 30: 197, 31: 185, 32: 197, 33: 198, 34: 195, 35: 210, 36: 195, 37: 213, 38: 182, 39: 182, 40: 205, 41: 214, 42: 178, 43: 207, 44: 201, 45: 203, 46: 209, 47: 172, 48: 195, 49: 214}, 'Super-sink': {}, 0: {50: 0, 51: 0, 52: 0, 53: 0, 54: 0, 55: 0, 56: 0, 57: 0, 58: 0, 59: 0, 60: 0, 61: 0, 62: 0, 63: 0, 64: 0, 65: 0, 66: 0, 67: 0, 68: 0, 69: 0, 70: 9, 71: 0, 72: 0, 73: 0, 74: 14, 75: 0, 76: 0, 77: 0, 78: 0, 79: 0, 80: 0, 81: 0, 82: 0, 83: 0, 84: 0, 85: 0, 86: 0, 87: 0, 88: 0, 89: 0, 90: 0, 91: 0, 92: 0, 93: 0, 94: 0, 95: 0, 96: 0, 97: 0, 98: 0, 99: 0, 100: 0, 101: 0, 102: 0, 103: 0, 104: 0, 105: 0, 106: 0, 107: 0, 108: 0, 109: 0, 110: 0, 111: 0, 112: 0, 113: 0, 114: 0, 115: 0, 116: 

In [29]:
# G2=modelSimplex(sources,sinks,edges)
# printGraphStats(G2,False)


Total Demand is:  0
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217,

In [30]:
# start = time.time()
# flowCost, flowDict = nx.network_simplex(G2)
# print("Cost is:",flowCost)
# end = time.time()
# print(end - start)
# print(flowDict)

Cost is: 3034933
0.9806149005889893
{0: {50: 0, 51: 0, 52: 0, 53: 0, 54: 0, 55: 0, 56: 0, 57: 0, 58: 0, 59: 0, 60: 0, 61: 0, 62: 0, 63: 0, 64: 0, 65: 0, 66: 0, 67: 0, 68: 0, 69: 0, 70: 9, 71: 0, 72: 0, 73: 0, 74: 14, 75: 0, 76: 0, 77: 0, 78: 0, 79: 0, 80: 0, 81: 0, 82: 0, 83: 0, 84: 0, 85: 0, 86: 0, 87: 0, 88: 0, 89: 0, 90: 0, 91: 0, 92: 0, 93: 0, 94: 0, 95: 0, 96: 0, 97: 0, 98: 0, 99: 0, 100: 0, 101: 0, 102: 0, 103: 0, 104: 0, 105: 0, 106: 0, 107: 0, 108: 0, 109: 0, 110: 0, 111: 0, 112: 0, 113: 0, 114: 0, 115: 0, 116: 0, 117: 0, 118: 0, 119: 0, 120: 0, 121: 0, 122: 0, 123: 0, 124: 0, 125: 0, 126: 0, 127: 0, 128: 0, 129: 0, 130: 0, 131: 0, 132: 0, 133: 0, 134: 0, 135: 0, 136: 0, 137: 0, 138: 11, 139: 0, 140: 0, 141: 0, 142: 0, 143: 0, 144: 0, 145: 0, 146: 0, 147: 0, 148: 0, 149: 0, 150: 0, 151: 0, 152: 0, 153: 0, 154: 0, 155: 0, 156: 0, 157: 0, 158: 0, 159: 0, 160: 0, 161: 0, 162: 0, 163: 0, 164: 0, 165: 0, 166: 0, 167: 0, 168: 0, 169: 0, 170: 0, 171: 0, 172: 0, 173: 0, 174: 0, 175: 0,

In [31]:
# tempList=[100,200,300,400,500,600,700,800,900,1000,1200,1400,1600,1800,2000]
# experiment=[]
# for i in tempList:
#   experiment+=[[50,i]]
#   experiment+=[[100,i]]
#   experiment+=[[300,i]]
#   experiment+=[[400,i]]
#   experiment+=[[500,i]]
#   experiment+=[[600,i]]
#   experiment+=[[700,i]]
# experiment.sort()
# print(experiment)

# # experiment=[[50,100],[100,300],[100,1200],[50,600],[200,600]]
# for event in experiment:
#   print("Event with Sources:",event[0],"Sinks:",event[1])
#   numberOfNodes=event[0]+event[1]
#   TotalCapacity=10000
#   TotalRequirement=10000
#   nodes=getNodes(numberOfNodes)
#   sources,sinks=distributeSourceSink(nodes,event[1])
#   sources=GetSources(sources,TotalCapacity)
#   sinks=GetSinks(sinks,TotalRequirement)
#   edges=getEdgesDummy(sources,sinks)
#   start = time.time()
#   G1=modelMaxFlow(sources,sinks,edges)
#   end = time.time()
#   print("Time to create Max Flow Model:",end - start)
#   start = time.time()
#   G2=modelSimplex(sources,sinks,edges)
#   end = time.time()
#   print("Time to create Simplex Model:",end - start)
#   start = time.time()
#   mincostFlow = nx.max_flow_min_cost(G1, "Super-source", "Super-sink")
#   # print("Cost is:", nx.cost_of_flow(G1, mincostFlow))
#   end = time.time()
#   print("Time to run Max Flow Model:",end - start)
#   start = time.time()
#   flowCost, flowDict = nx.network_simplex(G2)
#   # print("Cost is:",flowCost)
#   end = time.time()
#   print("Time to run Simplex Model:",end - start)
#   print("--------------------------------")
  



[[50, 100], [50, 200], [50, 300], [50, 400], [50, 500], [50, 600], [50, 700], [50, 800], [50, 900], [50, 1000], [50, 1200], [50, 1400], [50, 1600], [50, 1800], [50, 2000], [100, 100], [100, 200], [100, 300], [100, 400], [100, 500], [100, 600], [100, 700], [100, 800], [100, 900], [100, 1000], [100, 1200], [100, 1400], [100, 1600], [100, 1800], [100, 2000], [200, 100], [200, 200], [200, 300], [200, 400], [200, 500], [200, 600], [200, 700], [200, 800], [200, 900], [200, 1000], [200, 1200], [200, 1400], [200, 1600], [200, 1800], [200, 2000]]
Event with Sources: 50 Sinks: 100
Source Demand is: 10000
Sink Demand is: 10000
Time to create Max Flow Model: 0.014748096466064453
Total Demand is:  0
Time to create Simplex Model: 0.015515327453613281
Time to run Max Flow Model: 0.3116910457611084
Time to run Simplex Model: 0.059876441955566406
--------------------------------
Event with Sources: 50 Sinks: 200
Source Demand is: 10000
Sink Demand is: 10000
Time to create Max Flow Model: 0.018358945846

KeyboardInterrupt: ignored

In [37]:
# tempList=[100,200,300,400,500,600,700,800,900,1000,1200,1400,1600,1800,2000]
# experiment=[]
# for i in tempList:
#   experiment+=[[50,i]]
#   experiment+=[[100,i]]
#   experiment+=[[200,i]]
#   experiment+=[[300,i]]
#   experiment+=[[400,i]]
#   experiment+=[[500,i]]
# experiment.sort()
# print(experiment)
# header = ['sources', 'sinks', 'Time-create-MCMF', 'Time-create-simplex', 'Time-run-MCMF', 'Time-run-simplex','Time-run-cost-scaling']
# data=[]
# # experiment=[[50,100],[100,300],[100,1200],[50,600],[200,600]]
# for event in experiment:
#   row=[]
#   print("Event with Sources:",event[0],"Sinks:",event[1])
#   row+=[event[0],event[1]]
#   numberOfNodes=event[0]+event[1]
#   TotalCapacity=10000
#   TotalRequirement=10000
#   nodes=getNodes(numberOfNodes)
#   sources,sinks=distributeSourceSink(nodes,event[1])
#   sources=GetSources(sources,TotalCapacity)
#   sinks=GetSinks(sinks,TotalRequirement)
#   edges=getEdgesDummy(sources,sinks)
#   start = time.time()
#   G1=modelMaxFlow(sources,sinks,edges)
#   end = time.time()
#   row+=[end-start]
#   print("Time to create Max Flow Model:",end - start)
#   start = time.time()
#   G2=modelSimplex(sources,sinks,edges)
#   end = time.time()
#   row+=[end-start]
#   print("Time to create Simplex Model:",end - start)
#   start = time.time()
#   mincostFlow = nx.max_flow_min_cost(G1, "Super-source", "Super-sink")
#   # print("Cost is:", nx.cost_of_flow(G1, mincostFlow))
#   end = time.time()
#   row+=[end-start]
#   print("Time to run Max Flow Model:",end - start)
#   start = time.time()
#   flowCost, flowDict = nx.network_simplex(G2)
#   # print("Cost is:",flowCost)
#   end = time.time()
#   row+=[end-start]
#   start = time.time()
#   flowCost, flowDict = nx.capacity_scaling()(G2)
#   # print("Cost is:",flowCost)
#   end = time.time()
#   row+=[end-start]
#   print("Time to run Simplex Model:",end - start)
#   print("--------------------------------")
#   data+=[row]
#   print(row)
# with open('countries.csv', 'w', encoding='UTF8', newline='') as f:
#   writer = csv.writer(f)
#   writer.writerow(header)
#   writer.writerows(data)


[[300, 100], [300, 200], [300, 300], [300, 400], [300, 500], [300, 600], [300, 700], [300, 800], [300, 900], [300, 1000], [300, 1200], [300, 1400], [300, 1600], [300, 1800], [300, 2000], [400, 100], [400, 200], [400, 300], [400, 400], [400, 500], [400, 600], [400, 700], [400, 800], [400, 900], [400, 1000], [400, 1200], [400, 1400], [400, 1600], [400, 1800], [400, 2000]]
Event with Sources: 300 Sinks: 100
Source Demand is: 10000
Sink Demand is: 10000
Time to create Max Flow Model: 0.05040121078491211
Total Demand is:  0
Time to create Simplex Model: 0.05660724639892578
Time to run Max Flow Model: 3.7770633697509766
Time to run Simplex Model: 0.4285588264465332
--------------------------------
[300, 100, 0.05040121078491211, 0.05660724639892578, 3.7770633697509766, 0.4285588264465332]
Event with Sources: 300 Sinks: 200
Source Demand is: 10000
Sink Demand is: 10000
Time to create Max Flow Model: 0.10430574417114258
Total Demand is:  0
Time to create Simplex Model: 0.10538887977600098
Time

In [38]:
# print(data)

[[300, 100, 0.05040121078491211, 0.05660724639892578, 3.7770633697509766, 0.4285588264465332], [300, 200, 0.10430574417114258, 0.10538887977600098, 9.054643630981445, 0.8587954044342041], [300, 300, 0.15916776657104492, 0.16254925727844238, 17.83918833732605, 1.1939697265625], [300, 400, 0.21916437149047852, 0.23969626426696777, 18.169859886169434, 1.8260657787322998], [300, 500, 0.26867103576660156, 0.28411865234375, 24.879358291625977, 2.353260040283203], [300, 600, 0.33138036727905273, 0.35036683082580566, 31.504530668258667, 2.9480156898498535], [300, 700, 0.4062941074371338, 0.4034099578857422, 38.67970371246338, 3.5682318210601807], [300, 800, 0.455507755279541, 0.4611012935638428, 46.49650263786316, 3.797621488571167], [300, 900, 0.5293514728546143, 0.5224592685699463, 54.954914569854736, 4.538501501083374], [300, 1000, 0.5755939483642578, 0.5997312068939209, 64.07348251342773, 5.423325538635254], [300, 1200, 0.6901965141296387, 0.7009449005126953, 89.66243863105774, 5.633719682

In [39]:
# with open('test.csv', 'w', encoding='UTF8', newline='') as f:
#   writer = csv.writer(f)
#   writer.writerow(header)
#   writer.writerows(data)


In [41]:
# !cd drive/MyDrive/Network/


In [40]:
# 245544455ls

countries.csv  drive/  sample_data/  test.csv


[[50, 2000], [50, 2500], [50, 3000], [50, 3500], [50, 4000], [500, 100], [500, 200], [500, 300], [500, 400], [500, 500], [500, 600], [500, 700], [500, 800], [500, 900], [500, 1000], [500, 1200], [500, 1400], [500, 1600], [500, 1800], [500, 2000]]
Event with Sources: 50 Sinks: 2000
Source Demand is: 10000
Sink Demand is: 10000
Time to create Max Flow Model: 0.3131260871887207
Total Demand is:  0
Time to create Simplex Model: 0.3147697448730469
Time to run Max Flow Model: 16.080299854278564
Time to run Simplex Model: 2.037358522415161
--------------------------------
[50, 2000, 0.3131260871887207, 0.3147697448730469, 16.080299854278564, 2.037358522415161]
Event with Sources: 50 Sinks: 2500
Source Demand is: 10000
Sink Demand is: 10000
Time to create Max Flow Model: 0.23481035232543945
Total Demand is:  0
Time to create Simplex Model: 0.22924065589904785
Time to run Max Flow Model: 22.10248637199402
Time to run Simplex Model: 2.4659242630004883
--------------------------------
[50, 2500, 